In [50]:
import spotipy
import csv
import os
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from dotenv import load_dotenv

In [7]:
load_dotenv()

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

client_credentials_manager = SpotifyClientCredentials()
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [93]:
tracks = []
for i in range(2013, 2023):
    search_result = spotify.search(q=str(i), limit=50) #the result only consists of tracks

    tracks.extend(search_result['tracks']['items'])

    for i in range(19):
        search_result = spotify.next(search_result['tracks'])
        tracks.extend(search_result['tracks']['items'])

In [96]:
tracks_id = []
for i in range(len(tracks)):
    tracks_id.append(tracks[i]['id'])

tracks_id = list(dict.fromkeys(tracks_id))

In [97]:
track_features_list = []

for i in range(len(tracks_id)):
    track_features_list.append(spotify.audio_features(tracks_id[i]))

In [98]:
#there are some tracks without their audio features so we need to remove them from the list
track_features_list = list(filter(lambda x: x != [None], track_features_list))

In [99]:
for i in range(len(track_features_list)):
    track_features_list[i] = dict(list(filter(lambda x: x[0] == 'id' or type(x[1]) != str, list(track_features_list[i][0].items()))))

In [102]:
artists_id = []
for i in range(len(tracks)):
    for j in range(len(tracks[i]['artists'])):
        artists_id.append(tracks[i]['artists'][j]['id'])

artists_id = list(dict.fromkeys(artists_id))

In [103]:
artists = []
for i in range(len(artists_id)):
    artist_info = spotify.artist(artist_id=artists_id[i])
    cols = ['id', 'genres']
    artist = dict.fromkeys(cols)
    for key in artist:
        artist[key] = artist_info[key]
    artists.append(artist)

In [104]:
artists = list(filter(lambda x: x['genres'] != [] ,artists))

In [105]:
genres_list = []

for i in range(len(artists)):
    keys = ['id', 'genre']
    genre = dict.fromkeys(keys)
    for j in range(len(tracks)):
        if artists[i]['id'] == tracks[j]['artists'][0]['id']:
            genre['id'] = tracks[j]['id']
            genre['genre'] = artists[i]['genres'][0]
            genres_list.append(genre)

In [106]:
for i in range(len(track_features_list)):
    for j in range(len(genres_list)):
        if track_features_list[i]['id'] == genres_list[j]['id']:
            track_features_list[i]['genre'] = genres_list[j]['genre']

In [107]:
track_features_list = list(filter(lambda x: len(x) == 15, track_features_list)) #tracks having genre

In [ ]:
for i in range(len(track_features_list)):
    track_features_list[i] = {'id': track_features_list[i].pop('id'), **track_features_list[i]}

In [51]:
with open('audio_features.tsv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=track_features_list[0].keys(), delimiter='\t')
    writer.writeheader()
    for data in track_features_list:
        writer.writerow(data)

NameError: name 'track_features_list' is not defined

---------------------------------------------------------------------------------------------------------------

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('10_genres_audio_features.tsv', delimiter='\t')

In [3]:
df.head(10)

,id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genre
0,0VOMBffX1llDu37yQUt3u3,0.474,0.576,1,-6.963,1,0.2570,0.00248,0.000309,0.1740,0.285,157.947,195197,4,pop
1,2ZlABhxMMa43COmZ97kKsJ,0.799,0.575,6,-6.257,0,0.2820,0.22100,0.000000,0.1960,0.319,99.026,137151,4,pop
2,6uFn47ACjqYkc0jADwEdj1,0.772,0.639,1,-7.119,1,0.4670,0.15000,0.000000,0.0698,0.261,168.112,166560,4,pop
3,3lOMJTQTd6J34faYwASc33,0.795,0.859,2,-2.994,1,0.0542,0.03610,0.000000,0.0321,0.356,96.986,168107,4,pop
4,2g8HN35AnVGIk7B8yMucww,0.780,0.575,9,-7.247,0,0.2730,0.43000,0.000000,0.1430,0.773,84.492,252747,4,pop
5,4GssB27iJeqmfGxS94Tfij,0.461,0.897,3,-3.982,0,0.0920,0.00821,0.000000,0.0965,0.358,165.107,220537,4,pop
6,3wn8nOygkHLUQ9dlXM1rKW,0.705,0.613,7,-7.441,1,0.1230,0.26200,0.000007,0.1980,0.259,74.521,167187,4,pop
7,1jaTQ3nqY3oAAYyCTbIvnM,0.923,0.604,11,-6.671,0,0.2450,0.01700,0.000000,0.2720,0.826,145.062,139741,4,pop
8,6EDO9iiTtwNv6waLwa1UUq,0.800,0.560,5,-4.818,0,0.2610,0.05700,0.000000,0.1340,0.450,163.071,200221,4,pop
9,0MvGqcwwm0jnvIMAMjLhKw,0.612,0.580,0,-8.572,0,0.0419,0.52500,0.000414,0.2600,0.534,130.045,239560,4,pop


In [4]:
data = df.drop(['id'], axis=1)

In [5]:
label = data.iloc[:, -1]
label = label.sort_values()

In [6]:
words_list = []
for genre in label:
    words_list.extend(genre.split(' ')[-1:])
words_dict = dict.fromkeys(words_list)

for key in words_dict:
    words_dict[key] = words_list.count(key)

words_dict = dict(sorted(words_dict.items(), key=lambda words_dict: words_dict[1], reverse=True))

len(words_dict)

10

In [7]:
words_dict = dict(filter(lambda x: x[1] > 10, list(words_dict.items())))

In [8]:
words_dict.keys()

dict_keys(['country', 'hop', 'house', 'indie', 'metal', 'pop', 'edm', 'rap', 'rock', 'chillhop'])

In [9]:
flag = 0
for i in range(len(label)):
    for key in words_dict:
        if key in label[i]:
            label[i] = key
            flag = 1
    if flag == 0:
        label[i] = 'None'
    flag = 0

In [10]:
encoder = LabelEncoder()
y = encoder.fit_transform(label)
len(y)

9995

In [11]:
scaler = StandardScaler()
x = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
x.shape

(9995, 13)

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True, random_state=20)
x_train.shape

(6996, 13)

-------

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

import plotly.express as px

In [23]:
knn_model = KNeighborsClassifier()
dec_model = DecisionTreeClassifier()
mlp_model = MLPClassifier()
svm_model = SVC()
bagging_model = BaggingClassifier()
lgs_model = LogisticRegression()

In [24]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(knn_model, x_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
n_scores.mean()

0.361920226173445

In [25]:
lgs_model.fit(x_train, y_train)
knn_model.fit(x_train, y_train)
dec_model.fit(x_train, y_train)
mlp_model.fit(x_train, y_train)
svm_model.fit(x_train, y_train)
bagging_model.fit(x_train, y_train)

/home/sukivn/anaconda3/envs/min_ds-env/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:702: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



BaggingClassifier()

In [26]:
lgs_acc = lgs_model.score(x_test, y_test)
knn_acc = knn_model.score(x_test, y_test)
dec_acc = dec_model.score(x_test, y_test)
mlp_acc = mlp_model.score(x_test, y_test)
svm_acc = svm_model.score(x_test, y_test)
bagging_acc = bagging_model.score(x_test, y_test)

print("Logistic Regression Accuracy", lgs_acc)
print("K-Nearest-Neighbors Accuracy:", knn_acc)
print("Decision Tree Accuracy:", dec_acc)
print("Neural Network Accuracy:", mlp_acc)
print("Bagging Algorithms Accuracy:", bagging_acc)
print("Support Vector Machine Accuracy:", svm_acc)

Logistic Regression Accuracy 0.3781260420140047
K-Nearest-Neighbors Accuracy: 0.3561187062354118
Decision Tree Accuracy: 0.3677892630876959
Neural Network Accuracy: 0.41847282427475824
Bagging Algorithms Accuracy: 0.4401467155718573
Support Vector Machine Accuracy: 0.41580526842280763


In [27]:
accuracy_score(y_test, knn_model.predict(x_test))

0.3561187062354118

In [28]:
f1_score(y_test, knn_model.predict(x_test), average='weighted')

0.34115730951102385

In [29]:
fig = px.bar(
    x=["Logistic Regression", "K-Nearest-Neighbors", "Decision Tree", "Neural Network", "Bagging Algorithms", "Support Vector Machine"],
    y=[lgs_acc, knn_acc, dec_acc, mlp_acc, bagging_acc, svm_acc],
    color=["Logistic Regression", "K-Nearest-Neighbors", "Decision Tree", "Neural Network", "Bagging Algorithms", "Support Vector Machine"],
    labels={'x': "Model", 'y': "Accuracy"},
    title="Model Accuracy Comparison"
)

fig.show()